In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.display import display
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'plotly_white'
import logging
import logzero
logzero.loglevel(logging.INFO)

In [2]:
from dataclasses import dataclass
from typing import List, Tuple
from collections import Counter, defaultdict
import random
from copy import copy
import numpy as np
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from logzero import logger
import consed
from BITS.clustering.seq import ClusteringSeq
from BITS.seq.io import save_fasta
from BITS.seq.align import EdlibRunner
from BITS.seq.consed import ConsedRunner
from BITS.seq.utils import revcomp
from BITS.util.io import save_pickle, load_pickle
from BITS.util.proc import run_command, NoDaemonPool
from BITS.plot.plotly import make_line, make_hist, make_scatter, make_layout, show_plot
from vca.types import TRUnit

In [3]:
dir_fname = 'work/'
import os
os.chdir(dir_fname)

In [4]:
sync_reads = load_pickle("centromere_reads_incl_low_cover_rate_all_sync.pkl")

In [8]:
len(sync_reads)

1905

In [5]:
sync_reads_by_id = {read.id: read for read in sync_reads}

In [6]:
sync_reads_master = load_pickle("centromere_reads_incl_low_cover_rate_master.pkl")

In [7]:
len(sync_reads_master)

1665

In [10]:
sync_reads_by_id_master = {read.id: read for read in sync_reads_master}

## Load BLASTN overlaps

In [11]:
read_name_to_id = {read.name: read.id for read in sync_reads}
read_name_to_id_master = {read.name: read.id for read in sync_reads_master}

In [57]:
ovlps = []
with open("blast/ava.blastn", 'r') as f:
    for line in f:
        # NOTE: 1-indexed position! so, start -> start - 1, end -> end
        qseqid, sseqid, pident, length, _, _, qstart, qend, sstart, send, _, _ = line.strip().split('\t')
        length, qstart, qend, sstart, send = list(map(int, (length, qstart, qend, sstart, send)))
        
        if qseqid == sseqid:
            continue
        
        if length < 3000:
            continue

        qseqid, sseqid = read_name_to_id_master[qseqid], read_name_to_id_master[sseqid]
        qlen, slen = sync_reads_by_id_master[qseqid].length, sync_reads_by_id_master[sseqid].length
        
        strand = 'n' if sstart < send else 'c'
        if strand == 'c':
            sstart, send = slen - sstart, slen - send
            
        if qstart > 100 and sstart > 100:
            continue
        if qlen - qend > 100 and slen - send > 100:
            continue
            
        pdiff = 100. - float(pident)
        if pdiff >= 2.0:
            continue

        ovlps.append((qseqid, sseqid, strand, qstart, qend, qlen, sstart, send, slen, round(pdiff, 3)))   # NOTE: no #diffs

In [47]:
ovlps[:100]

[(73, 198167, 'n', 1, 9406, 13814, 2241, 11653, 11653, 0.159),
 (73, 16873, 'c', 7850, 13814, 13814, 1, 5974, 12959, 0.601),
 (73, 381787, 'c', 10101, 13814, 13814, 0, 3718, 12144, 0.188),
 (73, 317383, 'c', 10487, 13814, 13814, 1, 3328, 13424, 0.3),
 (300, 54493, 'n', 1, 4015, 13059, 8121, 12132, 12133, 0.125),
 (369, 436104, 'c', 8332, 12117, 12117, 0, 3787, 12858, 0.29),
 (369, 75008, 'c', 1, 3494, 12117, 9916, 13413, 13414, 0.771),
 (546, 273709, 'c', 1431, 12867, 12867, 0, 11432, 13101, 0.497),
 (546, 147506, 'n', 1, 7200, 12867, 5398, 12593, 12593, 0.25),
 (559, 299496, 'c', 1, 8778, 14494, 4545, 13298, 13299, 0.455),
 (559, 132531, 'c', 1, 8080, 14494, 4263, 12334, 12335, 0.148),
 (559, 202130, 'n', 9059, 14492, 14494, 1, 5416, 14534, 0.533),
 (741, 133575, 'c', 1, 4629, 12604, 6776, 11410, 11411, 0.474),
 (765, 356798, 'n', 486, 12783, 14235, 1, 12293, 12295, 0.398),
 (765, 248365, 'c', 2844, 14235, 14235, 0, 11399, 12256, 0.342),
 (875, 239252, 'n', 1, 11214, 12497, 726, 11945

In [48]:
len(ovlps)

2786

In [49]:
# 各リードの overlap の数; NOTE: ovlps are duplicated here
n_ovlps = Counter()   # {read_id: ovlp_count}
for ovlp in ovlps:
    n_ovlps[ovlp[0]] += 1
    n_ovlps[ovlp[1]] += 1

In [50]:
n_ovlps_for_each_read = [n_ovlps[read.id] // 2 for read in sync_reads_master]

In [51]:
show_plot([make_hist(n_ovlps_for_each_read, bin_size=1)])

In [52]:
def construct_string_graph(overlaps):
    nodes, edges = set(), set()
    for overlap in overlaps:
        f_id, g_id, strand, a_start, a_end, a_len, b_start, b_end, b_len, p_diff = overlap
        
        if strand == 'c':
            # a_read[a_start:a_end] ~~ strand(b_read[b_start:b_end]) となるようにする
            # b の座標は常に b_read を forward に見た時のもの。配列の切り出しが先。revcomp は後。
            b_start, b_end = b_len - b_end, b_len - b_start
            
        if a_start > 0:
            if strand == 'n':
                if b_end == b_len:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     --------->
                          g.B        g.E
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         ------------->
                                g.B           g.E
                    """
                    overlap_type = "suffix-prefix"
            else:
                if b_start == 0:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     <---------
                          g.E        g.B
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         <-------------
                                g.E           g.B
                    """
                    overlap_type = "suffix-suffix"
        else:
            if a_end == a_len:
                overlap_type = "contained"
            else:
                if strand == 'n':
                    """
                                f.B         f.E
                      f          ----------->
                      g    ----------->
                         g.B          g.E
                    """
                    if b_start == 0:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-suffix"
                else:
                    """
                                f.B         f.E
                      f          ----------->
                      g    <-----------
                         g.E           g.B
                    """
                    if b_end == b_len:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-prefix"

        if overlap_type in ["contains", "contained"]:
            continue
        elif overlap_type == "suffix-prefix":
            nodes.update(["%s:B" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:E" % g_id])
            edges.update([("%s:B" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:E" % g_id, b_len - b_end, p_diff)])
        elif overlap_type == "suffix-suffix":
            nodes.update(["%s:E" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:B" % g_id])
            edges.update([("%s:E" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:B" % g_id, b_start, p_diff)])
        elif overlap_type == "prefix-suffix":
            nodes.update(["%s:B" % f_id,
                          "%s:B" % g_id,
                          "%s:E" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:B" % g_id, b_start, p_diff),
                          ("%s:E" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])
        else:   # prefix-prefix
            nodes.update(["%s:B" % f_id,
                          "%s:E" % g_id,
                          "%s:B" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:E" % g_id, b_len - b_end, p_diff),
                          ("%s:B" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])

    return ig.Graph.DictList(edges=(dict(source=s, target=t, length=l, diff=p) for s, t, l, p in edges),
                             vertices=None,
                             directed=True)

def transitive_reduction(sg):
    v_mark = ["vacant" for v in sg.vs]
    e_reduce = {e.tuple: False for e in sg.es}
    FUZZ = 10   # in bp; this length is in general shorter than unit length, thus we accept no unit shifts

    for v in sg.vs:
        if v.outdegree() == 0:
            continue

        oes = sorted(sg.es.select(_source=v.index), key=lambda x: x["length"])
        longest = oes[-1]["length"] + FUZZ
        for oe in oes:
            v_mark[oe.target] = "inplay"

        for oe in oes:
            if v_mark[oe.target] == "inplay":
                ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
                for ooe in ooes:
                    if oe["length"] + ooe["length"] <= longest and v_mark[ooe.target] == "inplay":
                        v_mark[ooe.target] = "eliminated"

        for oe in oes:
            ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
            if len(ooes) > 1:
                shortest = ooes[0].target
                if v_mark[shortest] == "inplay":
                    v_mark[shortest] == "eliminated"
            for ooe in ooes:
                if ooe["length"] < FUZZ and v_mark[ooe.target] == "inplay":
                    v_mark[ooe.target] = "eliminated"

        for oe in oes:
            if v_mark[oe.target] == "eliminated":
                e_reduce[oe.tuple] = True   # TODO: confirm revcomp edges will be also removed in the same way
            v_mark[oe.target] = "vacant"

    # Re-construct a graph
    return ig.Graph.DictList(edges=(dict(source=e["source"],
                                         target=e["target"],
                                         length=e["length"],
                                         diff=e["diff"])
                                    for e in sg.es
                                    if not e_reduce[e.tuple]),
                             vertices=None,
                             directed=True)

def draw_graph(sg):
    E = [e.tuple for e in sg.es]
    N = sg.vcount()
    pos = sg.layout('kk')

    edge_trace = go.Scatter(x=[i for l in [(pos[s][0], pos[t][0], None) for s, t in E] for i in l],
                            y=[i for l in [(pos[s][1], pos[t][1], None) for s, t in E] for i in l],
                            line=dict(width=0.5, color='black'),
                            mode='lines')

    shapes = [make_line(pos[s][0] + (pos[t][0] - pos[s][0]) * 0.7,
                        pos[s][1] + (pos[t][1] - pos[s][1]) * 0.7,
                        pos[t][0],
                        pos[t][1],
                        "black",
                        4,
                        "below")
              for s, t in E]
    
    reads = [sync_reads_by_id[int(sg.vs[node]["name"].split(':')[0])] for node in range(N)]
    cover_rates = [sum([unit.end - unit.start for unit in read.units]) / read.length for read in reads]   # by 359-bp units

    node_trace = go.Scatter(x=[pos[node][0] for node in range(N)],
                            y=[pos[node][1] for node in range(N)],
                            text=[f"{sg.vs[node]['name']}<br>{round(cover_rates[node] * 100, 1)}% covered" for node in range(N)],
                            mode='markers',
                            marker=dict(
                                color=[cr for cr in cover_rates],
                                showscale=False,
                                colorscale='YlGnBu',
                                reversescale=False,
                                size=10,
                                line=dict(width=2)))

    layout = go.Layout(width=1000, height=1000,
                       xaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       yaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       shapes=shapes,
                       hovermode='closest',
                       margin=go.layout.Margin(l=0, r=0, b=0, t=0),
                       showlegend=False)
    fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
    py.iplot(fig)
    return pos

In [53]:
sg = transitive_reduction(construct_string_graph(ovlps))

In [54]:
cc = [(g, g.vcount()) for g in sg.clusters(mode="weak").subgraphs() if g.vcount() >= 10]

In [55]:
cc

[(<igraph.Graph at 0x2af628000318>, 21),
 (<igraph.Graph at 0x2af6280006d8>, 10),
 (<igraph.Graph at 0x2af6280007c8>, 112),
 (<igraph.Graph at 0x2af628000c78>, 10),
 (<igraph.Graph at 0x2af627ffc048>, 14),
 (<igraph.Graph at 0x2af627ffc318>, 13),
 (<igraph.Graph at 0x2af627ffc5e8>, 14),
 (<igraph.Graph at 0x2af627ffd408>, 14),
 (<igraph.Graph at 0x2af627ffd4f8>, 21),
 (<igraph.Graph at 0x2af627ffd5e8>, 17),
 (<igraph.Graph at 0x2af627ffdc78>, 11),
 (<igraph.Graph at 0x2af627ffa318>, 12),
 (<igraph.Graph at 0x2af627ffa408>, 11),
 (<igraph.Graph at 0x2af627ffae58>, 17),
 (<igraph.Graph at 0x2af627ff9318>, 11),
 (<igraph.Graph at 0x2af627ff6b88>, 11),
 (<igraph.Graph at 0x2af627ff6e58>, 11),
 (<igraph.Graph at 0x2af627ff5318>, 13),
 (<igraph.Graph at 0x2af627ff54f8>, 10),
 (<igraph.Graph at 0x2af6280104f8>, 10),
 (<igraph.Graph at 0x2af6280106d8>, 10),
 (<igraph.Graph at 0x2af628010a98>, 13),
 (<igraph.Graph at 0x2af62800e048>, 10),
 (<igraph.Graph at 0x2af62800e138>, 17),
 (<igraph.Graph

In [56]:
for g, s in cc:
    draw_graph(g)

In [45]:
!(cat blast/ava.sh)

#!/bin/bash
#$ -N sleep
#$ -o sge.log
#$ -j y
#$ -S /bin/bash
#$ -cwd
#$ -V
#$ -q all.q
#$ -pe smp 28

#makeblastdb -in centromere_reads_incl_low_cover_rate_master.fasta -dbtype nucl -out reads -parse_seqids

blastn -num_threads 28 -outfmt 6 -perc_identity 97 -db reads -query centromere_reads_incl_low_cover_rate_master.fasta -out ava.blastn


なぜ master 持ちはうまくオーバーラップが取れない？タンデムリピートのせいでアラインメントがずれている？STR でもないのにそんなことは。。？